### ชื่อ-นามสกุล : ณิชาภัทร บุญพาวัฒนา
### รหัสนิสิต : 6241072726

Data Source : https://www.homepro.co.th/c/APP0904

In [ ]:
# import packages and module
import pandas as pd
import requests
from bs4 import BeautifulSoup
from PIL import Image
import io

In [ ]:
print(f'pandas version = {pd.__version__}')

### Web scraping

In [ ]:
url = 'https://www.homepro.co.th/c/APP0904'
r=requests.get(url)

In [ ]:
r.text[:200]

In [ ]:
s=BeautifulSoup(r.text, 'lxml')
type(s)

In [ ]:
print(s.prettify())

In [ ]:
all_products = s.select('div.hp-card-product-plp')
all_products

In [ ]:
all_products[0].find('img')

In [ ]:
all_products[0].find('img')['alt']

In [ ]:
all_products[0].find('img')['src']

In [ ]:
all_products[0].select_one('a')['href']

In [ ]:
t = all_products[0]

r=requests.get(t.select_one('img')['src'])
img = Image.open(io.BytesIO(r.content))
img

In [ ]:
# link ของรูปในเว็บไซต์
pic = []
for e in all_products:
    prod_img=e.select_one('img')['src']
    pic.append(prod_img)
    
pic[:3]

In [ ]:
lst1=[]
lst2=[]
lst3=[]

for e in all_products:
    lst1.append(e.select_one('img')['alt'])
    lst2.append(e.select_one('img')['src'])
    lst3.append(e.select_one('a')['href'])

In [ ]:
r = []
for e in all_products:
    price1 = e.find('div', {'class': 'product-info-container'}).find('span', 'reduced-price')
    if price1 is not None:
        r.append(price1.get_text())
    elif price1 is None:
        price2 = e.find('div', {'class': 'product-info-container'}).find('div', 'original-price')        
        if price2 is not None:
            r.append(price2.get_text())

In [ ]:
d = []
for e in all_products:
    discount = e.find('div', {'class': 'product-info-container'}).find('div', 'online-discount')
    if discount is not None:
        d.append(discount.get_text())
    else:
        d.append(discount)

d[:5]

In [ ]:
j = list(zip(lst1, lst2, lst3, r, d))

### Convert to DataFrame

In [ ]:
pd.options.display.max_colwidth = 0

In [ ]:
df=pd.DataFrame(j, columns=['prod_name', 'img_url', 'link', 'price', 'discount'])
df.head()

In [ ]:
df['discount'] = df['discount'].str.replace(r'\nซื้อออนไลน์ ลดเพิ่ม', '', regex=True)
df['discount'] = df['discount'].str.replace(r'บาท\n', '', regex=True)
df['price'] = df['price'].str.replace(r'บาท', '', regex=True)
df['price'] = df['price'].str.replace(r',', '', regex=True)

In [ ]:
df['discount'].fillna(value=0, inplace=True)
df['discount'] = df['discount'].astype(int)
df['price'] = df['price'].astype(int)

In [ ]:
df['net_price'] = df['price']-df['discount']

### Convert to HTML

In [ ]:
from IPython.display import HTML

In [ ]:
def to_html_img(url, alt=None, width=100):
    if alt==None:
        alt=''
    return f'<img alt="{alt}" src="{url}" width="{width}"/>'


In [ ]:
df['img']=df.apply(lambda r: to_html_img(r['img_url'], r['prod_name'], 100), axis=1)

In [ ]:
HTML(df[:5][['prod_name', 'img']].to_html(escape=False))

In [ ]:
def to_html_a(url, text):
    if text==None:
        text=''
    return f'<a href="{url}">{text}</a>'

In [ ]:
df['name_link']=df.apply(lambda r: to_html_a(r['link'], r['prod_name']), axis=1)

In [ ]:
HTML(df[:5][['prod_name', 'img', 'name_link']].to_html(escape=False))

In [ ]:
df['img_link']=df.apply(lambda r: to_html_a(r['link'], r['img']), axis=1)

In [ ]:
HTML(df[:5][['prod_name', 'img', 'name_link', 'img_link']].to_html(escape=False))

In [ ]:
d_html = HTML(df[:5][['name_link', 'img_link', 'net_price']].to_html(escape=False))
d_html

In [ ]:
df.head()

### Convert to Excel

In [ ]:
d_excel = df[['prod_name', 'img', 'net_price']]
d_excel.to_excel('Catalog.xlsx')